In [3]:
import soundfile as sf
import librosa
import numpy as np
import matplotlib.pyplot as plt
import scipy.fft as sp
from numpy import fft

In [4]:
# Take in data path
# Return [spoof list, bonafide list]
dev = True

def create_bf_spf_lists(path):
    f = open(path)
    spoof_list = []
    bonafide_list = []
    for line in f:
        tokens = line.strip().split()
        if tokens[3] == 'spoof':
            spoof_list.append(tokens[1])
        else:
            bonafide_list.append(tokens[1])
    return [spoof_list, bonafide_list]

[spoof_listf, bonafide_listf] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.dev.female.trl.txt")
[spoof_listm, bonafide_listm] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.dev.male.trl.txt")

#[spoof_listf, bonafide_listf] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.eval.female.trl.txt")
#[spoof_listm, bonafide_listm] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.eval.male.trl.txt")

In [5]:
def extract_time_data(list, num):
    time_data = []
    #fspoof = open(path, "w")
    for i in range(num):
        dpath = 'DS_10283_3336\LA\LA\ASVspoof2019_LA_dev\\flac\\' + list[i] + '.flac'
        #dpath = 'DS_10283_3336\LA\LA\ASVspoof2019_LA_eval\\flac\\' + list[i] + '.flac'
        x, sr = librosa.load(dpath, sr=44100)
        time_data.append(x)
    return (time_data, sr)

ftime_data_spoof, sr = extract_time_data(spoof_listf, 4500)
ftime_data_bonafide, sr = extract_time_data(bonafide_listf, 1500)
mtime_data_spoof, sr = extract_time_data(spoof_listm, 4500)
mtime_data_bonafide, sr = extract_time_data(bonafide_listm, 1500)

In [9]:
def get_max_length(data):
    return max(len(vector) for vector in data)

def pad_or_truncate(data, max_length):
    padded_data = []
    for vector in data:
        if len(vector) < max_length:
            pad_width = max_length - len(vector)
            padded_vector = np.pad(vector, pad_width=(0, pad_width), mode='constant')
        elif len(vector) > max_length:
            padded_vector = vector[:max_length]
        else:
            padded_vector = vector
        padded_data.append(padded_vector)
    return padded_data

# Extract mfccs from time_data with padding
def extract_cqcc(time_data, max_length):
    padded_data = pad_or_truncate(time_data, max_length)
    
    cqcc_data = []
    
    for x in padded_data:
        cqcc = librosa.cqt(y=x, sr=16000, hop_length=16384, n_bins=30)
        logC = np.abs(cqcc)
        librosa.feature.mfcc(S=logC)
        # Flatten to a large vector

        flattened_vector = np.array(logC).flatten()
        cqcc_data.append(flattened_vector)
        
        #cqcc_data.append(logC)
    
    return np.array(cqcc_data)

In [10]:
fmax_length = max(get_max_length(ftime_data_spoof), get_max_length(ftime_data_bonafide))
mmax_length = max(get_max_length(mtime_data_spoof), get_max_length(mtime_data_bonafide))
max_length = max(fmax_length, mmax_length)

cqcc_spooff = extract_cqcc(ftime_data_spoof, max_length)
cqcc_bonafidef = extract_cqcc(ftime_data_bonafide, max_length)

cqcc_spoofm = extract_cqcc(mtime_data_spoof, max_length)
cqcc_bonafidem = extract_cqcc(mtime_data_bonafide, max_length)

In [11]:
np.set_printoptions(threshold=np.inf, precision=10, linewidth=np.inf, suppress=True)

# Open files for writing
with open("spoof_data_cqcc_male.txt", "w") as fspoofw:
    for row in cqcc_spoofm:
        array_str = np.array_str(row)
        fspoofw.write(array_str + '\n')

with open("bonafide_data_cqcc_male.txt", "w") as fbonafidew:
    for row in cqcc_bonafidem:
        array_str = np.array_str(row)
        fbonafidew.write(array_str + '\n')

with open("spoof_data_cqcc_female.txt", "w") as fspoofw:
    for row in cqcc_spooff:
        array_str = np.array_str(row)
        fspoofw.write(array_str + '\n')

with open("bonafide_cqcc_female.txt", "w") as fbonafidew:
    for row in cqcc_bonafidef:
        array_str = np.array_str(row)
        fbonafidew.write(array_str + '\n')